In [26]:
# Install required packages (only needed once per environment)
!pip install -q -r requirements.txt

# Initialize Otter and import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import otter

# Initialize Otter grader
grader = otter.Notebook('dungeon_lab.ipynb')

# Optional plot styling for consistent visuals
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (8, 4)
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['lines.linewidth'] = 2

np.random.seed(42)


# 🧙‍♂️ Probability Quest: The Dungeon of Convergence

Welcome to the **Dungeon of Convergence**, traveler! Rumor says there are infinitely many rooms hidden beneath the Earth — each one containing a treasure chest. Some rooms are generous, others barren, and some are cursed. You can open one chest per room, forever. In this lab you will predict whether you’ll keep finding treasures forever or if your luck will eventually run dry, understand how your average loot behaves as you go deeper, and reveal the mathematical forces guiding your fate: the Borel–Cantelli Lemmas and different modes of convergence.


## Learning Objectives

- Apply the Borel–Cantelli Lemmas to infinite random events.
- Empirically distinguish almost sure, in probability, and distribution convergence.
- Understand how independence, dependence, and tail behavior affect long-term outcomes.
- Visualize running averages and observe convergence behavior in different dungeons.


## Dungeon Setup
In each room *n*, treasure appears with probability *pₙ*.  
If treasure appears, you gain **1 loot point** for that room; otherwise, you gain **0**.  
We model *Xₙ* as a Bernoulli random variable:

**Xₙ = { 1 if treasure is found in room n; 0 otherwise }**

Your **cumulative treasure** after *n* rooms is:

**Sₙ = Σᵢ₌₁ⁿ Xᵢ**


Your cumulative treasure after \(n\) rooms is:
### Dungeons

| Dungeon | Loot probability (pₙ) | Notes |
|----------|------------------------|--------|
| **Finite**  | pₙ = n⁻¹·³ | Finitely many hits (almost surely) |
| **Endless** | pₙ = 1/n | Infinitely many hits (rarer) |
| **Cursed**  | pₙ = 1/n (dependent) | Correlated streaks |
| **Greedy**  | Xₙ = n · 1(Aₙ),  P(Aₙ) = 1/n | Rare jackpots, heavy tails |

Different dungeon setups yield different *limits* and *modes of convergence*.  
Later, you’ll connect these ideas to the **Strong and Weak Laws of Large Numbers (SLLN/WLLN)**  
and compare convergence **almost surely**, **in probability**, and **in distribution**.

## 1. A Single Run Through the Dungeon

To get started, you'll implement a helper function **`simulate_dungeon`** that simulates a sequence of treasure rooms.
Given a function **p_n** specifying the probability of finding treasure in room **n** and a total number of rooms **N**, this
function should return two sequences:

- **X₁, X₂, …, Xₙ** — indicators for whether treasure was found in each room (0 or 1)
- **S₁, S₂, …, Sₙ** — cumulative treasure totals up to each room

You'll use this function throughout the rest of the lab.


In [27]:
def simulate_dungeon(p_func, N, seed=None, constant=1):
    '''
    Simulate a random dungeon with N rooms using loot probabilities given by p_func.

    Parameters:
        p_func (callable): A function taking a 1-based room index and returning p_n.
        N (int): Number of rooms to simulate.

    Returns:
        tuple: (X, S) where X is a numpy array of 0/1 indicators and S is the cumulative sum of X.
    '''
    if seed is not None: 
        np.random.seed(seed)
    

    X=[]
    S=[]
    cum_sum=0
    for n in range(1, N+1): 
        sample = np.random.rand()
        if sample < p_func(n): 
            X.append(1)
            cum_sum+=1
        else: 
            X.append(0)
        S.append(cum_sum)
    
    return (np.array(X), np.array(S))
    


In [28]:
grader.check('q1')

q1 results: All test cases passed!

## 2. Finite Dungeon

In the **Finite Dungeon**, the loot probability follows **pₙ = n⁻¹·³**.  
According to the *Borel–Cantelli Lemma*, only finitely many treasures are expected to appear (almost surely).

Below, implement a function **`finite_final_loot`** that simulates this dungeon for **N = 1000** rooms (using seed `42`)  
and returns the final cumulative treasure **S₁₀₀₀**.

This number should be small — typically just a handful of treasures.


In [29]:
def finite_final_loot():
    '''
    Simulate the Finite dungeon for N=1000 rooms with seed=42 and p_n = n^{-1.3}.

    Returns:
        float: The final cumulative treasure after 1000 rooms.
    '''
    
    return simulate_dungeon(lambda n: n**(-1.3), 1000, 42)[1][-1]


In [30]:
grader.check('q2')

q2 results: All test cases passed!

## 3. Endless Dungeon

In the **Endless Dungeon**, the loot probability follows **pₙ = 1/n**.  
According to the *Borel–Cantelli Lemma*, you will continue to find treasure infinitely often — but it becomes rarer over time.

Write a function **`endless_final_loot`** that simulates **N = 1000** rooms (using seed `42`)  
and returns the final cumulative treasure **S₁₀₀₀**.

Even though treasure appears infinitely often in theory, the first 1000 rooms may still yield only a small number of successes.


In [31]:
def endless_final_loot():
    '''
    Simulate the Endless dungeon for N=1000 rooms with seed=42 and p_n = 1/n.

    Returns:
        float: The final cumulative treasure after 1000 rooms.
    '''
    return simulate_dungeon(lambda n: 1/n, 1000, 42)[1][-1]


In [32]:
grader.check('q3')

q3 results: All test cases passed!

## 4. Cursed Dungeon

In the **Cursed Dungeon**, the loot probability still follows **pₙ = 1/n**,  
but outcomes are **dependent** across rooms. When you find treasure, your luck carries forward —  
your chance of finding loot in the next room roughly **doubles** (up to a maximum of 1).  
If you miss, your luck falters, and your next chance is **cut in half**.  

This creates alternating streaks of fortune and drought that violate the independence assumptions  
behind the Borel–Cantelli lemmas.  

Implement a function **`cursed_final_loot`** that simulates this dependent process for **N = 1000** rooms
and returns the final cumulative treasure **S₁₀₀₀**.


In [33]:

def cursed_final_loot():
    '''
    Simulate the Cursed dungeon for N=1000 rooms (seed=42) with dependent loot events.

    Returns:
        float: The final cumulative treasure after 1000 rooms.
    '''
    np.random.seed(42)
    cum_sum=0   
    p_factor = 1
    for n in range(1, 1001): 
        sample = np.random.rand()
        if sample < min(1, p_factor*1/n): 
            cum_sum+=1
            p_factor=2
        else: 
            p_factor=1/2
    return cum_sum


In [34]:
grader.check('q4')

q4 results: All test cases passed!

## 5. Greedy Dungeon

The **Greedy Dungeon** features extremely heavy tails:  
**Xₙ = n · 1(Aₙ)** where **P(Aₙ) = 1/n**.  
In other words, when treasure appears in room **n**, you collect **n loot points** instead of just 1.

Implement a function **`greedy_loot`** to simulate **N = 1000** rooms (using seed `42`)  
and return a tuple `(total_loot, average_loot)` where:

- **total_loot** is the final cumulative treasure  
- **average_loot** is **total_loot / 1000**


In [35]:
def greedy_loot():
    '''
    Simulate the Greedy dungeon for N=1000 rooms with seed=42. Each treasure event awards the room number.

    Returns:
        tuple: (total_loot, average_loot) where average_loot = total_loot / 1000.
    '''
    np.random.seed(42)
    cum_sum=0   
    for n in range(1, 1001): 
        sample = np.random.rand()
        if sample < 1/n: 
            cum_sum+=n
    return cum_sum, cum_sum/1000


In [36]:
grader.check('q5')

q5 results: All test cases passed!

## 6. Running Average Convergence

Define the **running average** of your cumulative loot as:

**Yₙ = Sₙ / n**

This measures the *average loot per room* as you progress through the dungeon.

In this section, you'll simulate and visualize **Yₙ** for each dungeon type.  
The behavior of the running average reveals how different notions of convergence manifest in practice:

- When a theorem guarantees **almost sure convergence**, every individual run should visibly stabilize.  
- When convergence is only **in probability**, most runs will settle, but some will fluctuate indefinitely.  
- In **heavy-tailed** settings, the average may fail to converge even in distribution, producing erratic trajectories.

Implement the following functions, each returning the array of running averages  
**{Y₁, Y₂, …, Y₁₀₀₀}** for the corresponding dungeon:

1. **`finite_running_avg()`** – simulate the Finite Dungeon and compute the running averages.  
2. **`endless_running_avg()`** – simulate the Endless Dungeon and compute the running averages.  
3. **`cursed_running_avg()`** – simulate the Cursed Dungeon (with dependence) and compute the running averages.  
4. **`greedy_running_avg()`** – simulate the Greedy Dungeon and compute the running averages.

Each function should also produce a simple line plot of **Yₙ** versus **n**,  
illustrating how the average behaves as the number of rooms grows.

In [37]:
def finite_running_avg():
    '''
    Simulate the finite dungeon for N=1000 rooms (seed=42) and return the running averages Y_n = S_n/n.

    Returns:
        numpy.ndarray: An array of length 1000 containing the running averages.
    '''
    _, S = simulate_dungeon(lambda n: n**(-1.3), 1000)
    n = np.arange(1,len(S)+1)
    Y= S/n
    # plt.plot(n, Y)
    return Y






In [38]:
grader.check('q6-1')

q6-1 results: All test cases passed!

In [39]:
def endless_running_avg():
    '''
    Simulate the endless dungeon for N=1000 rooms (seed=42) and return the running averages Y_n = S_n/n.

    Returns:
        numpy.ndarray: An array of length 1000 containing the running averages.
    '''
    
    _, S = simulate_dungeon(lambda n: 1/n, 1000)
    n = np.arange(1,len(S)+1)
    Y = S/n
    # plt.plot(n,S)
    return Y
        



In [40]:
grader.check('q6-2')

q6-2 results: All test cases passed!

In [41]:
def greedy_running_avg():
    '''
    Simulate the greedy dungeon for N=1000 rooms (seed=42) and return the running averages Y_n = S_n/n.

    Returns:
        numpy.ndarray: An array of length 1000 containing the running averages.
    '''
    Y=[]
    np.random.seed(42)
    cum_sum=0   
    for n in range(1, 1001): 
        sample = np.random.rand()
        if sample < 1/n: 
            cum_sum+=n
        Y.append(cum_sum/n)
    X=np.linspace(1,1000,1000)
    # plt.plot(X,Y)
    return np.array(Y)

    


In [42]:
grader.check('q6-3')

q6-3 results: All test cases passed!


### Cursed Dungeon Running Average

Next, compute the running average for the Cursed Dungeon. Define
\[Y_n = 

rac{S_n}{n}\]
where \(S_n\) is the cumulative treasure up to room \(n\). Plot and return an array of length 1000 containing the running averages. We'll use the same dependent dynamics as above.


In [43]:

def cursed_running_avg():
    '''
    Simulate the Cursed dungeon for N=1000 rooms (seed=42) and return the running averages Y_n = S_n/n.

    Returns:
        numpy.ndarray: An array of length 1000 containing the running averages.
    '''
    Y=[]
    np.random.seed(42)
    cum_sum=0   
    p_factor = 1
    for n in range(1, 1001): 
        sample = np.random.rand()
        if sample < min(1, p_factor*1/n): 
            cum_sum+=1
            p_factor=2
        else: 
            p_factor=1/2
        Y.append(cum_sum/n)
    X=np.linspace(1,1000,1000)
    # plt.plot(X,Y)
    return np.array(Y)


In [44]:
grader.check('q6-4')

q6-4 results: All test cases passed!

## 7. Modes of Convergence
## 7. Modes of Convergence

Summarize how the running averages converge in each dungeon.

Implement a function **`classify_convergence`** that returns a dictionary mapping each dungeon to the modes of convergence that apply.  
Possible modes are: **"almost surely"**, **"in probability"**, and **"in distribution"**.

For example:
- The **Finite Dungeon** converges only in *A*
- The **Endless Dungeon** has *no guarantee*.   
- The **Cursed Dungeon** (with dependence) converges in *B* and *C*.  
- The **Greedy Dungeon** converges in *D*, *E* and *F*.

Return a dictionary like:

```python
{
    "Finite": ["A"],
    "Endless": [],
    "Cursed": ["B", "C"],
    "Greedy": ["D", "E", "F"],
}



In [55]:
def classify_convergence():
    '''
    Return a dictionary mapping each dungeon to the applicable modes of convergence.

    Returns:
        dict: Keys are "Finite", "Endless", "Cursed", and "Greedy", values are lists of modes.
    '''
    return {"Finite":["almost surely", "in distribution"], "Endless":["in probability"], "Cursed":[], "Greedy":["in probability"]}


In [56]:
grader.check('q7')

q7 results: All test cases passed!

## 8. Written Questions

In this final section, reflect on the mathematical ideas behind the dungeons. Write your responses in complete sentences, drawing on the definitions, simulations, and plots you've seen.


<!-- BEGIN QUESTION -->

**Written Question 1:** Compare and contrast almost sure convergence and convergence in probability in the context of the Finite and Endless dungeons. How do your simulation results and running average plots illustrate the differences?  
**solution:**
This all comes down to long run behavior. Notice that if we continued to run the finite simulation over a mulititude of random seeds, we'd reproduce the same convergence to 0 and the same exponential decay shape. However, for the Endless simulation, it converges in probability because we are guaranteed that the probability gets increasingly smaller as n increases, thus less loot and a convergent value. However, it is not entirely guaranteed that there is a small chance of there being a room for n after a long time, such that there is loot there. 

We can see the differences manifest in the graph as well. For the finite simulation, as described previously, we can see the exponential decay shape that converges to 0 everytime. On the other hand, for the Endless dungeon simulation, it has this staircase like shape that eventually converges to some value. It is important to note that the value the Endless dungeon converges to is never the same over repeated runs with random seed—whereas the Finite dungeon does. The convergence to a specific value over the convergence at some point is ultimately the differences between the mathematical ideas that describe almost sure convergence and convergence in probability. 

Feel free to use this space to draft your answer. When you finish, remember to paste your response and any relevant plots/figures **into the accompanying Gradescope assignment**.


<!-- END QUESTION -->


<!-- BEGIN QUESTION -->

**Written Question 2:** The Greedy Dungeon exhibits heavy tails and fails to converge in distribution. Explain why heavy tails break distribution convergence even if the running averages appear to stabilize. What does this tell us about interpreting simulation results in heavy-tailed settings?


Feel free to use this space to draft your answer. When you finish, remember to paste your response and any relevant plots/figures **into the accompanying Gradescope assignment**.
**solution** 
Heavy tails break distribution because there is always an incredibly small chance—nonetheless, a chance still exists—that the running average will spike up at some n down the line, given the design of the greedy algorithim. This sporadic behavior makes it difficult to even capture an interval of convergence between some small delta, as the spikes can be incredibly large. 

Moreover, when interpreting simulation results in heavy-tailed settings, we must take into consideration that the mathematical design behind the algorithim is increedibly important for reasoning convergence. If the algorithim has a probability to oscillate with amplitudes incredibly high, it's nearly impossible to capture an interval of convergence, and can then become uninterpretable for a definitive convergent value. 


<!-- END QUESTION -->


<!-- BEGIN QUESTION -->

**Written Question 3:** The Cursed Dungeon involves dependent loot events. How might dependence among events break the assumptions of the Borel–Cantelli lemmas? Provide an example of how correlated events could affect the occurrence of infinitely many treasures, and what type of convergence (if any) you might expect.


Feel free to use this space to draft your answer. When you finish, remember to paste your response and any relevant plots/figures **into the accompanying Gradescope assignment**.


<!-- END QUESTION -->


## 8. Conclusion

In this lab you've explored several infinite sequences of random variables and seen how the Borel–Cantelli lemmas and different modes of convergence predict your long-term fate. You simulated *finite* and *endless* dungeons where treasure becomes vanishingly rare, a *cursed* dungeon with dependencies, and a *greedy* dungeon with heavy tails. Running averages can converge in different senses: almost surely, in probability, or in distribution. Use these distinctions to understand when and how randomness settles in real systems.

For your submission, export this notebook onto a zip file with the command below!

In [48]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True,pdf=False)

Running your submission against local test cases...


Your submission received the following results when run against available test cases:

    q1 results: All test cases passed!

    q2 results: All test cases passed!

    q3 results: All test cases passed!

    q4 results: All test cases passed!

    q5 results: All test cases passed!

    q6-1 results: All test cases passed!

    q6-2 results: All test cases passed!

    q6-3 results: All test cases passed!

    q6-4 results: All test cases passed!

    q7 results: All test cases passed!

    q8 results:
        convergence targets result:
            ❌ Test case failed
            Error at line 13 in test q8:
                  result = convergence_targets()
            TypeError: NoneType' object is not callable
